In [1]:
sqlContext = SQLContext(sc)

## Load house prices

In [2]:
text_RDD = sc.textFile("/data/houses.txt")

In [3]:
def mapper_parse_lines(line):
    """Parse line into (neighborhoood, price) pair"""
    words = line.split()
    return (words[1], float(words[2]), int(words[0]))

In [4]:
house_prices_RDD = text_RDD.map(mapper_parse_lines)

In [5]:
house_prices_RDD.collect()

[(u'Downtown', 400000.0, 3),
 (u'Downtown', 240000.0, 2),
 (u'Hilltop', 650000.0, 3)]

In [6]:
house_prices_df = sqlContext.createDataFrame(house_prices_RDD,
                ["neighborhood", "price", "bedrooms"])

In [7]:
house_prices_df.show()

+------------+--------+--------+
|neighborhood|   price|bedrooms|
+------------+--------+--------+
|    Downtown|400000.0|       3|
|    Downtown|240000.0|       2|
|     Hilltop|650000.0|       3|
+------------+--------+--------+



In [8]:
house_prices_df.printSchema()

root
 |-- neighborhood: string (nullable = true)
 |-- price: double (nullable = true)
 |-- bedrooms: long (nullable = true)



## Load inflation

In [9]:
inflation_text_RDD = sc.textFile("/data/inflation.txt")

In [10]:
def mapper_parse__inflation_lines(line):
    """Parse line into (neighborhoood, inflation) pair"""
    words = line.split()
    return (words[0], float(words[1]))

In [11]:
inflation_RDD = inflation_text_RDD.map(mapper_parse__inflation_lines)

In [12]:
inflation_RDD.collect()

[(u'Downtown', 2.1), (u'Hilltop', 4.5)]

In [15]:
inflation_df = sqlContext.createDataFrame(inflation_RDD,
                        ["neighborhood", "inflation"])

In [16]:
inflation_df.printSchema()

root
 |-- neighborhood: string (nullable = true)
 |-- inflation: double (nullable = true)



## join

In [18]:
house_prices_df.join(inflation_df).show()

+------------+--------+--------+------------+---------+
|neighborhood|   price|bedrooms|neighborhood|inflation|
+------------+--------+--------+------------+---------+
|    Downtown|400000.0|       3|    Downtown|      2.1|
|    Downtown|240000.0|       2|    Downtown|      2.1|
|    Downtown|400000.0|       3|     Hilltop|      4.5|
|    Downtown|240000.0|       2|     Hilltop|      4.5|
|     Hilltop|650000.0|       3|    Downtown|      2.1|
|     Hilltop|650000.0|       3|     Hilltop|      4.5|
+------------+--------+--------+------------+---------+



In [19]:
def mapper_multiply_price_inflation(row):
    return (row["neighborhood"], row["price"] * row["inflation"])

In [23]:
house_prices_nextyear_join = house_prices_df.join(inflation_df, "neighborhood").map(mapper_multiply_price_inflation)

In [24]:
house_prices_nextyear_df = \
    house_prices_nextyear_join.select(
        "neighborhood", 
        (house_prices_nextyear_join.price * (1+house_prices_nextyear_join.inflation/100.)).alias("price"),
        "bedrooms"
    )

AttributeError: 'PipelinedRDD' object has no attribute 'select'

In [22]:
house_prices_nextyear_df.show()

NameError: name 'house_prices_nextyear_df' is not defined

## reduce

In [25]:
house_prices_nextyear_df.groupBy("neighborhood").sum("price").show()

NameError: name 'house_prices_nextyear_df' is not defined

## Exercise

Find the **mean** price for next year by number of bedrooms